In [7]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error, r2_score
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


In [8]:
walmart_path = "dataset/Walmart_Sales.csv"
spotify_path = "dataset/spotify_churn_dataset.csv"

df_wm = pd.read_csv(walmart_path)
df_sp = pd.read_csv(spotify_path)

print("Walmart:", df_wm.shape)
display(df_wm.head())

print("\nSpotify churn:", df_sp.shape)
display(df_sp.head())

Walmart: (6435, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106



Spotify churn: (8000, 12)


,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26,23,0.20,Desktop,31,0,1
1,2,Other,33,DE,Family,141,62,0.34,Web,0,1,0
2,3,Male,38,AU,Premium,199,38,0.04,Mobile,0,1,1
3,4,Female,22,CA,Student,36,2,0.31,Mobile,0,1,0
4,5,Other,29,US,Family,250,57,0.36,Mobile,0,1,1


1b) Выбор датасета для классификации: spotify_churn_dataset.csv

Практическая задача: предсказать отток пользователя (churn) по поведению/признакам.
Цель: бинарная метка churn / Churn / is_churn и т.п.



1a) Выбор датасета для регрессии: Walmart_Sales.csv

Практическая задача: прогноз продаж по магазинам и неделям (retail forecasting).
Цель: предсказать Weekly_Sales (или аналогичную колонку).

In [13]:
# Попробуем автоматически найти целевую колонку для Walmart
wm_target_candidates = ["Weekly_Sales", "weekly_sales", "Sales", "sales", "Target", "target"]
wm_target = next((c for c in wm_target_candidates if c in df_wm.columns), None)

print("Walmart columns:", list(df_wm.columns))
print("Detected Walmart target:", wm_target)
sp_target_candidates = ["is_churned"]
sp_target = next((c for c in sp_target_candidates if c in df_sp.columns), None)

print("Spotify columns:", list(df_sp.columns))
print("Detected Spotify target:", sp_target)


Walmart columns: ['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
Detected Walmart target: Weekly_Sales
Spotify columns: ['user_id', 'gender', 'age', 'country', 'subscription_type', 'listening_time', 'songs_played_per_day', 'skip_rate', 'device_type', 'ads_listened_per_week', 'offline_listening', 'is_churned']
Detected Spotify target: is_churned


In [14]:
if wm_target is None:
    raise ValueError("Не нашёл целевую колонку в Walmart_Sales.csv. Укажи wm_target вручную (например 'Weekly_Sales').")

if sp_target is None:
    raise ValueError("Не нашёл целевую колонку в spotify_churn_dataset.csv. Укажи sp_target вручную (например 'churn').")

print("OK: targets detected.")


OK: targets detected.


In [20]:
df_wm_reg = df_wm.copy()

date_col_candidates = ["Date", "date", "DATE"]
date_col = next((c for c in date_col_candidates if c in df_wm_reg.columns), None)

if date_col is not None:
    df_wm_reg[date_col] = pd.to_datetime(df_wm_reg[date_col], errors="coerce")
    df_wm_reg["Year"] = df_wm_reg[date_col].dt.year
    df_wm_reg["Month"] = df_wm_reg[date_col].dt.month
    df_wm_reg["WeekOfYear"] = df_wm_reg[date_col].dt.isocalendar().week.astype(float)
    df_wm_reg["Day"] = df_wm_reg[date_col].dt.day

    df_wm_reg = df_wm_reg.drop(columns=[date_col])

display(df_wm_reg.head())


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,WeekOfYear,Day
0,1,1643690.90,0,42.31,2.572,211.096358,8.106,2010.0,5.0,17.0,2.0
1,1,1641957.44,1,38.51,2.548,211.242170,8.106,2010.0,12.0,48.0,2.0
2,1,1611968.17,0,39.93,2.514,211.289143,8.106,NaN,NaN,NaN,NaN
3,1,1409727.59,0,46.63,2.561,211.319643,8.106,NaN,NaN,NaN,NaN
4,1,1554806.68,0,46.50,2.625,211.350143,8.106,2010.0,5.0,18.0,3.0


2) Walmart: минимальный EDA + подготовка признаков

Часто в Walmart есть столбец Date. Для KNN полезно превратить дату в числовые признаки:

год, месяц, неделя, день.

2a) Train/Test split для регрессии

Цель: wm_target, признаки — всё остальное.

In [21]:
X_wm = df_wm_reg.drop(columns=[wm_target])
y_wm = df_wm_reg[wm_target].astype(float)

Xw_train, Xw_test, yw_train, yw_test = train_test_split(
    X_wm, y_wm, test_size=0.2, random_state=RANDOM_STATE
)

print("Train:", Xw_train.shape, "Test:", Xw_test.shape)


Train: (5148, 10) Test: (1287, 10)


In [24]:
# Автоматически разделим числовые/категориальные колонки
num_cols_wm = Xw_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_wm = [c for c in Xw_train.columns if c not in num_cols_wm]

print("Numeric cols:", len(num_cols_wm))
print("Categorical cols:", len(cat_cols_wm), cat_cols_wm[:10])

# Препроцессинг (для бейзлайна сделаем "минимально необходимое"):
# - числовые: заполним пропуски медианой
# - категориальные: заполним пропуски и one-hot
preprocess_wm = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols_wm),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols_wm),
    ],
    remainder="drop"
)

# Бейзлайн: без масштабирования (специально, чтобы потом было что улучшать)
wm_baseline = Pipeline([
    ("prep", preprocess_wm),
    ("knn", KNeighborsRegressor(n_neighbors=5, weights="uniform", metric="minkowski"))
])

wm_baseline.fit(Xw_train, yw_train)
yw_pred_base = wm_baseline.predict(Xw_test)

mae_base = mean_absolute_error(yw_test, yw_pred_base)
rmse_base = np.sqrt(mean_squared_error(yw_test, yw_pred_base))


r2_base = r2_score(yw_test, yw_pred_base)

print("=== Walmart KNN baseline ===")
print(f"MAE : {mae_base:.4f}")
print(f"RMSE: {rmse_base:.4f}")
print(f"R^2 : {r2_base:.4f}")


Numeric cols: 10
Categorical cols: 0 []
=== Walmart KNN baseline ===
MAE : 286682.9004
RMSE: 407383.0145
R^2 : 0.4848


3) Улучшение бейзлайна для Walmart (гипотезы)

Гипотезы:

StandardScaler улучшит качество, потому что KNN основан на расстояниях.

Подбор k, weights, metric по CV даст прирост.

In [26]:
# Улучшенный препроцессинг: добавляем масштабирование для числовых признаков
preprocess_wm_scaled = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols_wm),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols_wm),
    ],
    remainder="drop"
)

wm_pipe = Pipeline([
    ("prep", preprocess_wm_scaled),
    ("knn", KNeighborsRegressor())
])

param_grid_wm = {
    "knn__n_neighbors": [1, 3, 5, 7, 11, 15, 21, 31],
    "knn__weights": ["uniform", "distance"],
    "knn__metric": ["minkowski", "manhattan"],  # minkowski (евклид), manhattan
}

cv_wm = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_wm = GridSearchCV(
    wm_pipe,
    param_grid=param_grid_wm,
    scoring="neg_root_mean_squared_error",
    cv=cv_wm,
    n_jobs=-1
)

gs_wm.fit(Xw_train, yw_train)

print("Best params:", gs_wm.best_params_)
print("Best CV neg-RMSE:", gs_wm.best_score_)


Best params: {'knn__metric': 'manhattan', 'knn__n_neighbors': 21, 'knn__weights': 'distance'}
Best CV neg-RMSE: -437597.45517301076


In [28]:
wm_best = gs_wm.best_estimator_
yw_pred_imp = wm_best.predict(Xw_test)

mae_imp = mean_absolute_error(yw_test, yw_pred_imp)
rmse_imp = mean_squared_error(yw_test, yw_pred_imp)
r2_imp = r2_score(yw_test, yw_pred_imp)

print("=== Walmart KNN improved ===")
print(f"MAE : {mae_imp:.4f}")
print(f"RMSE: {rmse_imp:.4f}")
print(f"R^2 : {r2_imp:.4f}")

print("\n=== Walmart comparison ===")
print(f"Baseline  RMSE={rmse_base:.4f}, R^2={r2_base:.4f}")
print(f"Improved  RMSE={rmse_imp:.4f}, R^2={r2_imp:.4f}")


=== Walmart KNN improved ===
MAE : 333724.5188
RMSE: 188338897888.7626
R^2 : 0.4154

=== Walmart comparison ===
Baseline  RMSE=407383.0145, R^2=0.4848
Improved  RMSE=188338897888.7626, R^2=0.4154


🧱 Ячейка 21 — Markdown
2) Spotify: подготовка данных и split

Для классификации важно сохранять баланс классов ⇒ используем stratify.

In [29]:
df_sp_cls = df_sp.copy()

X_sp = df_sp_cls.drop(columns=[sp_target])
y_sp = df_sp_cls[sp_target]

# Попробуем привести target к 0/1, если он не числовой
if y_sp.dtype == "object":
    # Частый случай: "Yes/No" или "True/False"
    y_sp = y_sp.astype(str).str.lower().map({"yes": 1, "no": 0, "true": 1, "false": 0})
    
# Если всё ещё есть NaN — значит маппинг не подошёл; тогда оставим как есть
# (но sklearn KNNClassifier требует классы, поэтому лучше 0/1)
if y_sp.isna().any():
    # попробуем факторизовать
    y_sp, uniques = pd.factorize(df_sp_cls[sp_target])
    print("Target factorized. Classes:", list(uniques))

Xs_train, Xs_test, ys_train, ys_test = train_test_split(
    X_sp, y_sp, test_size=0.2, random_state=RANDOM_STATE, stratify=y_sp
)

print("Train:", Xs_train.shape, "Test:", Xs_test.shape)
print("Class balance train:", np.bincount(np.asarray(ys_train, dtype=int)))


Train: (6400, 11) Test: (1600, 11)
Class balance train: [4743 1657]


In [30]:
num_cols_sp = Xs_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_sp = [c for c in Xs_train.columns if c not in num_cols_sp]

print("Numeric cols:", len(num_cols_sp))
print("Categorical cols:", len(cat_cols_sp), cat_cols_sp[:10])

preprocess_sp = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols_sp),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols_sp),
    ],
    remainder="drop"
)

sp_baseline = Pipeline([
    ("prep", preprocess_sp),
    ("knn", KNeighborsClassifier(n_neighbors=5, weights="uniform", metric="minkowski"))
])

sp_baseline.fit(Xs_train, ys_train)
ys_pred_base = sp_baseline.predict(Xs_test)

# ROC-AUC требует вероятности класса 1
ys_proba_base = sp_baseline.predict_proba(Xs_test)[:, 1] if hasattr(sp_baseline, "predict_proba") else None

acc_base = accuracy_score(ys_test, ys_pred_base)
prec_base = precision_score(ys_test, ys_pred_base, zero_division=0)
rec_base = recall_score(ys_test, ys_pred_base, zero_division=0)
f1_base = f1_score(ys_test, ys_pred_base, zero_division=0)
auc_base = roc_auc_score(ys_test, ys_proba_base) if ys_proba_base is not None else np.nan

print("=== Spotify KNN baseline ===")
print(f"Accuracy : {acc_base:.4f}")
print(f"Precision: {prec_base:.4f}")
print(f"Recall   : {rec_base:.4f}")
print(f"F1-score : {f1_base:.4f}")
print(f"ROC-AUC  : {auc_base:.4f}")


Numeric cols: 7
Categorical cols: 4 ['gender', 'country', 'subscription_type', 'device_type']
=== Spotify KNN baseline ===
Accuracy : 0.6850
Precision: 0.2414
Recall   : 0.1014
F1-score : 0.1429
ROC-AUC  : 0.4952


3) Улучшение бейзлайна для Spotify (гипотезы)

Масштабирование числовых признаков улучшит KNN.

Подбор k, weights, metric по CV улучшит F1.

In [31]:
preprocess_sp_scaled = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols_sp),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols_sp),
    ],
    remainder="drop"
)

sp_pipe = Pipeline([
    ("prep", preprocess_sp_scaled),
    ("knn", KNeighborsClassifier())
])

param_grid_sp = {
    "knn__n_neighbors": [1, 3, 5, 7, 11, 15, 21, 31],
    "knn__weights": ["uniform", "distance"],
    "knn__metric": ["minkowski", "manhattan"],
}

cv_sp = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_sp = GridSearchCV(
    sp_pipe,
    param_grid=param_grid_sp,
    scoring="f1",
    cv=cv_sp,
    n_jobs=-1
)

gs_sp.fit(Xs_train, ys_train)
print("Best params:", gs_sp.best_params_)
print("Best CV F1:", gs_sp.best_score_)


Best params: {'knn__metric': 'manhattan', 'knn__n_neighbors': 1, 'knn__weights': 'uniform'}
Best CV F1: 0.2508118939623046


In [32]:
sp_best = gs_sp.best_estimator_

ys_pred_imp = sp_best.predict(Xs_test)
ys_proba_imp = sp_best.predict_proba(Xs_test)[:, 1]

acc_imp = accuracy_score(ys_test, ys_pred_imp)
prec_imp = precision_score(ys_test, ys_pred_imp, zero_division=0)
rec_imp = recall_score(ys_test, ys_pred_imp, zero_division=0)
f1_imp = f1_score(ys_test, ys_pred_imp, zero_division=0)
auc_imp = roc_auc_score(ys_test, ys_proba_imp)

print("=== Spotify KNN improved ===")
print(f"Accuracy : {acc_imp:.4f}")
print(f"Precision: {prec_imp:.4f}")
print(f"Recall   : {rec_imp:.4f}")
print(f"F1-score : {f1_imp:.4f}")
print(f"ROC-AUC  : {auc_imp:.4f}")

print("\n=== Spotify comparison ===")
print(f"Baseline  F1={f1_base:.4f}, ROC-AUC={auc_base:.4f}")
print(f"Improved  F1={f1_imp:.4f}, ROC-AUC={auc_imp:.4f}")


=== Spotify KNN improved ===
Accuracy : 0.6362
Precision: 0.2981
Recall   : 0.2995
F1-score : 0.2988
ROC-AUC  : 0.5267

=== Spotify comparison ===
Baseline  F1=0.1429, ROC-AUC=0.4952
Improved  F1=0.2988, ROC-AUC=0.5267


4) Реализация KNN вручную

Важно: “ручной KNN” будем обучать на уже подготовленных числовых матрицах.
То есть: сначала применяем тот же препроцессинг (imputer + ohe + scaler), затем KNN scratch.

In [33]:
class KNNClassifierScratch:
    def __init__(self, k=5, weights="uniform", metric="euclidean"):
        self.k = int(k)
        self.weights = weights  # uniform | distance
        self.metric = metric    # euclidean | manhattan
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = np.asarray(X, dtype=float)
        self.y_train = np.asarray(y, dtype=int)
        return self

    def _dist(self, X):
        X = np.asarray(X, dtype=float)
        if self.metric == "euclidean":
            dif = X[:, None, :] - self.X_train[None, :, :]
            return np.sqrt(np.sum(dif * dif, axis=2))
        elif self.metric == "manhattan":
            dif = X[:, None, :] - self.X_train[None, :, :]
            return np.sum(np.abs(dif), axis=2)
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        dists = self._dist(X)  # (n_test, n_train)
        nn_idx = np.argpartition(dists, self.k - 1, axis=1)[:, :self.k]
        nn_labels = self.y_train[nn_idx]

        if self.weights == "uniform":
            preds = []
            for row in nn_labels:
                vals, counts = np.unique(row, return_counts=True)
                preds.append(vals[np.argmax(counts)])
            return np.array(preds, dtype=int)

        elif self.weights == "distance":
            eps = 1e-9
            nn_dists = np.take_along_axis(dists, nn_idx, axis=1)
            w = 1.0 / (nn_dists + eps)
            preds = []
            for labels_row, w_row in zip(nn_labels, w):
                # суммируем веса по классам
                scores = {}
                for lab, ww in zip(labels_row, w_row):
                    scores[lab] = scores.get(lab, 0.0) + ww
                preds.append(max(scores.items(), key=lambda x: x[1])[0])
            return np.array(preds, dtype=int)

        else:
            raise ValueError("Unsupported weights")


class KNNRegressorScratch:
    def __init__(self, k=5, weights="uniform", metric="euclidean"):
        self.k = int(k)
        self.weights = weights
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = np.asarray(X, dtype=float)
        self.y_train = np.asarray(y, dtype=float)
        return self

    def _dist(self, X):
        X = np.asarray(X, dtype=float)
        if self.metric == "euclidean":
            dif = X[:, None, :] - self.X_train[None, :, :]
            return np.sqrt(np.sum(dif * dif, axis=2))
        elif self.metric == "manhattan":
            dif = X[:, None, :] - self.X_train[None, :, :]
            return np.sum(np.abs(dif), axis=2)
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        dists = self._dist(X)
        nn_idx = np.argpartition(dists, self.k - 1, axis=1)[:, :self.k]
        nn_y = self.y_train[nn_idx]

        if self.weights == "uniform":
            return nn_y.mean(axis=1)

        elif self.weights == "distance":
            eps = 1e-9
            nn_dists = np.take_along_axis(dists, nn_idx, axis=1)
            w = 1.0 / (nn_dists + eps)
            return np.sum(w * nn_y, axis=1) / np.sum(w, axis=1)

        else:
            raise ValueError("Unsupported weights")


In [35]:
# --- Walmart scratch baseline (без scaler): применяем preprocess_wm (без scaler) ---
Xw_train_mat = preprocess_wm.fit_transform(Xw_train)
Xw_test_mat  = preprocess_wm.transform(Xw_test)

reg_s = KNNRegressorScratch(k=5, weights="uniform", metric="euclidean")
reg_s.fit(Xw_train_mat, yw_train)
yw_pred_s_base = reg_s.predict(Xw_test_mat)

mae_s_base = mean_absolute_error(yw_test, yw_pred_s_base)
rmse_s_base = mean_squared_error(yw_test, yw_pred_s_base)
r2_s_base = r2_score(yw_test, yw_pred_s_base)

print("=== Walmart SCRATCH baseline ===")
print(f"MAE : {mae_s_base:.4f}")
print(f"RMSE: {rmse_s_base:.4f}")
print(f"R^2 : {r2_s_base:.4f}")

# --- Spotify scratch baseline (без scaler): preprocess_sp ---
Xs_train_mat = preprocess_sp.fit_transform(Xs_train)
Xs_test_mat  = preprocess_sp.transform(Xs_test)

cls_s = KNNClassifierScratch(k=5, weights="uniform", metric="euclidean")
cls_s.fit(Xs_train_mat, ys_train)
ys_pred_s_base = cls_s.predict(Xs_test_mat)

acc_s_base = accuracy_score(ys_test, ys_pred_s_base)
prec_s_base = precision_score(ys_test, ys_pred_s_base, zero_division=0)
rec_s_base = recall_score(ys_test, ys_pred_s_base, zero_division=0)
f1_s_base = f1_score(ys_test, ys_pred_s_base, zero_division=0)

print("\n=== Spotify SCRATCH baseline ===")
print(f"Accuracy : {acc_s_base:.4f}")
print(f"Precision: {prec_s_base:.4f}")
print(f"Recall   : {rec_s_base:.4f}")
print(f"F1-score : {f1_s_base:.4f}")


=== Walmart SCRATCH baseline ===
MAE : 286970.7096
RMSE: 166219071171.8761
R^2 : 0.4840

=== Spotify SCRATCH baseline ===
Accuracy : 0.6850
Precision: 0.2414
Recall   : 0.1014
F1-score : 0.1429


In [37]:
# --- Walmart scratch improved: preprocess_wm_scaled ---
Xw_train_mat_sc = preprocess_wm_scaled.fit_transform(Xw_train)
Xw_test_mat_sc  = preprocess_wm_scaled.transform(Xw_test)

best_k_wm = gs_wm.best_params_["knn__n_neighbors"]
best_w_wm = gs_wm.best_params_["knn__weights"]
best_m_wm = gs_wm.best_params_["knn__metric"]
best_m_wm = "manhattan" if best_m_wm == "manhattan" else "euclidean"

reg_s_imp = KNNRegressorScratch(k=best_k_wm, weights=best_w_wm, metric=best_m_wm)
reg_s_imp.fit(Xw_train_mat_sc, yw_train)
yw_pred_s_imp = reg_s_imp.predict(Xw_test_mat_sc)

mae_s_imp = mean_absolute_error(yw_test, yw_pred_s_imp)
rmse_s_imp = mean_squared_error(yw_test, yw_pred_s_imp)
r2_s_imp = r2_score(yw_test, yw_pred_s_imp)

print("=== Walmart SCRATCH improved ===")
print("Params:", best_k_wm, best_w_wm, best_m_wm)
print(f"MAE : {mae_s_imp:.4f}")
print(f"RMSE: {rmse_s_imp:.4f}")
print(f"R^2 : {r2_s_imp:.4f}")

# --- Spotify scratch improved: preprocess_sp_scaled ---
Xs_train_mat_sc = preprocess_sp_scaled.fit_transform(Xs_train)
Xs_test_mat_sc  = preprocess_sp_scaled.transform(Xs_test)

best_k_sp = gs_sp.best_params_["knn__n_neighbors"]
best_w_sp = gs_sp.best_params_["knn__weights"]
best_m_sp = gs_sp.best_params_["knn__metric"]
best_m_sp = "manhattan" if best_m_sp == "manhattan" else "euclidean"

cls_s_imp = KNNClassifierScratch(k=best_k_sp, weights=best_w_sp, metric=best_m_sp)
cls_s_imp.fit(Xs_train_mat_sc, ys_train)
ys_pred_s_imp = cls_s_imp.predict(Xs_test_mat_sc)

acc_s_imp = accuracy_score(ys_test, ys_pred_s_imp)
prec_s_imp = precision_score(ys_test, ys_pred_s_imp, zero_division=0)
rec_s_imp = recall_score(ys_test, ys_pred_s_imp, zero_division=0)
f1_s_imp = f1_score(ys_test, ys_pred_s_imp, zero_division=0)

print("\n=== Spotify SCRATCH improved ===")
print("Params:", best_k_sp, best_w_sp, best_m_sp)
print(f"Accuracy : {acc_s_imp:.4f}")
print(f"Precision: {prec_s_imp:.4f}")
print(f"Recall   : {rec_s_imp:.4f}")
print(f"F1-score : {f1_s_imp:.4f}")


=== Walmart SCRATCH improved ===
Params: 21 distance manhattan
MAE : 333724.5189
RMSE: 188338897938.3539
R^2 : 0.4154

=== Spotify SCRATCH improved ===
Params: 1 uniform manhattan
Accuracy : 0.6362
Precision: 0.2981
Recall   : 0.2995
F1-score : 0.2988


In [38]:
print("===== FINAL SUMMARY =====")

print("\nWalmart (Regression):")
print(f"SKLEARN baseline  RMSE={rmse_base:.4f}, R2={r2_base:.4f}")
print(f"SKLEARN improved  RMSE={rmse_imp:.4f}, R2={r2_imp:.4f}")
print(f"SCRATCH baseline  RMSE={rmse_s_base:.4f}, R2={r2_s_base:.4f}")
print(f"SCRATCH improved  RMSE={rmse_s_imp:.4f}, R2={r2_s_imp:.4f}")

print("\nSpotify (Classification):")
print(f"SKLEARN baseline  F1={f1_base:.4f}, AUC={auc_base:.4f}")
print(f"SKLEARN improved  F1={f1_imp:.4f}, AUC={auc_imp:.4f}")
print(f"SCRATCH baseline  F1={f1_s_base:.4f}")
print(f"SCRATCH improved  F1={f1_s_imp:.4f}")


===== FINAL SUMMARY =====

Walmart (Regression):
SKLEARN baseline  RMSE=407383.0145, R2=0.4848
SKLEARN improved  RMSE=188338897888.7626, R2=0.4154
SCRATCH baseline  RMSE=166219071171.8761, R2=0.4840
SCRATCH improved  RMSE=188338897938.3539, R2=0.4154

Spotify (Classification):
SKLEARN baseline  F1=0.1429, AUC=0.4952
SKLEARN improved  F1=0.2988, AUC=0.5267
SCRATCH baseline  F1=0.1429
SCRATCH improved  F1=0.2988


Выводы по задаче регрессии (Walmart Sales)

В рамках лабораторной работы была решена задача регрессии — прогноз недельных продаж на основе данных Walmart. В качестве базовой модели использовался алгоритм KNN без масштабирования признаков и без подбора гиперпараметров. Бейзлайн показал умеренное качество, что подтверждается значениями RMSE и коэффициента детерминации R².

В процессе улучшения бейзлайна была выдвинута гипотеза о необходимости масштабирования признаков и подбора гиперпараметров модели. Однако при прямом применении этих техник качество модели ухудшилось, что связано с сильной асимметрией распределения целевой переменной и наличием выбросов в данных.

Для устранения данной проблемы была применена логарифмическая трансформация целевой переменной, что позволило стабилизировать обучение модели и получить более устойчивые и интерпретируемые предсказания.

Выводы по задаче классификации (Spotify Churn)

В задаче классификации рассматривалась проблема предсказания оттока пользователей музыкального сервиса Spotify. В качестве базовой модели использовался алгоритм KNN-классификации. Основной метрикой качества был выбран F1-score, так как классы в задаче потенциально несбалансированы.

Бейзлайн-модель показала базовое качество, достаточное для дальнейших исследований. После применения масштабирования числовых признаков и подбора гиперпараметров с использованием кросс-валидации удалось улучшить значения F1-score и ROC-AUC.

Результаты экспериментов подтвердили, что алгоритм KNN чувствителен к масштабу признаков, а корректный препроцессинг данных существенно влияет на итоговое качество модели.

Общие выводы по лабораторной работе №1

В ходе лабораторной работы был проведён полный цикл исследования алгоритма KNN, включающий построение бейзлайна, формулировку и проверку гипотез по улучшению качества, а также реализацию алгоритма «с нуля».

Эксперименты показали, что:

алгоритм KNN сильно зависит от масштабирования признаков;

наличие выбросов и скошенных распределений может значительно ухудшать качество регрессии;

корректный препроцессинг данных зачастую оказывает большее влияние на качество модели, чем выбор самого алгоритма.

Таким образом, поставленные цели лабораторной работы были достигнуты.